# Computer Vision 2023 Assignment 2: Image matching and retrieval

In this prac, you will experiment with image feature detectors, descriptors and matching. There are 3 main parts to the prac:

- matching an object in a pair of images
- searching for an object in a collection of images
- analysis and discussion of results

## General instructions

As before, you will use this notebook to run your code and display your results and analysis. Again we will mark a PDF conversion of your notebook, referring to your code if necessary, so you should ensure your code output is formatted neatly. 

***When converting to PDF, include the outputs and analysis only, not your code.*** You can do this from the command line using the `nbconvert` command (installed as part of Jupyter) as follows:

    jupyter nbconvert Assignment2.ipynb --to pdf --no-input --TagRemovePreprocessor.remove_cell_tags 'remove-cell'

This will also remove the preamble text from each question. It has been packaged into a small notebook you can run in colab, called notebooktopdf.ipynb


We will use the `OpenCV` library to complete the prac. It has several built in functions that will be useful. You are expected to consult documentation and use them appropriately.

As with the last assignment it is somewhat up to you how you answer each question. Ensure that the outputs and report are clear and easy to read so that the markers can rapidly assess what you have done, why, and how deep is your understanding. This includes:

- sizing, arranging and captioning image outputs appropriately
- explaining what you have done clearly and concisely
- clearly separating answers to each question

## Data

We have provided some example images for this assignment, available through a link on the MyUni assignment page. The images are organised by subject matter, with one folder containing images of book covers, one of museum exhibits, and another of urban landmarks. You should copy these data into a directory A2_smvs, keeping the directory structure the same as in the zip file.  

Within each category (within each folder), there is a “Reference” folder containing a clean image of each object and a “Query” folder containing images taken on a mobile device. Within each category, images with the same name contain the same object (so 001.jpg in the Reference folder contains the same book as 001.jpg in the Query folder).
The data is a subset of the Stanford Mobile Visual Search Dataset which is available at 

<http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/index.html>. 

The full data set contains more image categories and more query images of the objects we have provided, which may be useful for your testing!

Do not submit your own copy of the data or rename any files or folders! For marking, we will assume the datasets are available in subfolders of the working directory using the same folder names provided.

Here is some general setup code, which you can edit to suit your needs.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
""" !pwd
%cd drive/MyDrive/Colab\ Notebooks
!pwd """

In [ ]:
# Numpy is the main package for scientific computing with Python. 
import numpy as np
import cv2

# Matplotlib is a useful plotting library for python 
import matplotlib.pyplot as plt
# This code is to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots, can be changed
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
def draw_outline(ref, query, model):
    """
        Draw outline of reference image in the query image.
        This is just an example to show the steps involved.
        You can modify to suit your needs.
        Inputs:
            ref: reference image
            query: query image
            model: estimated transformation from query to reference image
    """
    h,w = ref.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,model)

    img = query.copy()
    cv2.polylines(img,[np.int32(dst)],True,255,3, cv2.LINE_AA)

    plt.imshow(img, 'gray'), plt.show()

def draw_inliers(img1, img2, kp1, kp2, matches, matchesMask):
    """
        Draw inlier between images
        img1 / img2: reference/query  img
        kp1 / kp2: their keypoints
        matches : list of (good) matches after ratio test
        matchesMask: Inlier mask returned in cv2.findHomography() 
    """
    matchesMask = matchesMask.ravel().tolist()
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = 2)
    img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches,None,**draw_params)
    plt.imshow(img3, 'gray'),plt.show()


# Question 1: Matching an object in a pair of images (60%)


In this question, the aim is to accurately locate a reference object in a query image, for example:

![Books](book.png "Books")

0. Download and read through the paper [ORB: an efficient alternative to SIFT or SURF](https://www.researchgate.net/publication/221111151_ORB_an_efficient_alternative_to_SIFT_or_SURF) by Rublee et al. You don't need to understand all the details, but try to get an idea of how it works. ORB combines the FAST corner detector (covered in week 3) and the BRIEF descriptor. BRIEF is based on similar ideas to the SIFT descriptor we covered week 3, but with some changes for efficiency.

1. [Load images] Load the first (reference, query) image pair from the "book_covers" category using opencv (e.g. `img=cv2.imread()`). Check the parameter option in "
cv2.imread()" to ensure that you read the gray scale image, since it is necessary for computing ORB features.

2. [Detect features] Create opencv ORB feature extractor by `orb=cv2.ORB_create()`. Then you can detect keypoints by `kp = orb.detect(img,None)`, and compute descriptors by `kp, des = orb.compute(img, kp)`. You need to do this for each image, and then you can use `cv2.drawKeypoints()` for visualization.

3. [Match features] As ORB is a binary feature, you need to use HAMMING distance for matching, e.g., `bf = cv2.BFMatcher(cv2.NORM_HAMMING)`. Then you are requried to do KNN matching (k=2) by using `bf.knnMatch()`. After that, you are required to use "ratio_test" to find good matches. By default, you can set `ratio=0.8`.

4. [Plot and analyze] You need to visualize the matches by using the `cv2.drawMatches()` function. Also you can change the ratio values, parameters in `cv2.ORB_create()`, and distance functions in `cv2.BFMatcher()`. Please discuss how these changes influence the match numbers.


In [ ]:
# load images as grey scale
img1 = cv2.imread('A2_smvs/book_covers/Reference/001.jpg', 0)
if not np.shape(img1):
  # Error message and print current working dir
  print("Could not load img1. Check the path, filename and current working directory\n")
  !pwd
img2 = cv2.imread("A2_smvs/book_covers/Query/001.jpg", 0)
if not np.shape(img2):
  # Error message and print current working dir
  print("Could not load img2. Check the path, filename and current working directory\n")
  !pwd

In [ ]:
# Your code for descriptor matching tests here

# compute detector and descriptor, see (2) above
orb = cv2.ORB_create()


# find the keypoints and descriptors with ORB, see (2) above
kp1=orb.detect(img1,None)
kp1,des1 = orb.compute(img1,kp1)
kp2=orb.detect(img2,None)
kp2,des2 = orb.compute(img2,kp2)



# draw keypoints, see (2) above
kp1img = cv2.drawKeypoints(img1,kp1, 0, (0, 0, 255),
                                 flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

plt.imshow(kp1img), plt.show()

kp2img= cv2.drawKeypoints(img2,kp2, 0, (0, 0, 255),
                                 flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)


plt.imshow(kp2img), plt.show()
# create BFMatcher object, see (3) above

bf = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=False)
# Match descriptors, see (3) above
matches = bf.knnMatch(des1,des2,2)


# Apply ratio test, see (3) above
good = []
for m,n in matches:
    if m.distance < 0.8*n.distance:
        good.append(m)


# draw matches, see (4) above
img3= cv2.drawMatches(img1, kp1, img2, kp2, np.random.choice(good,100),None,flags=2)
plt.imshow(img3), plt.show()



***Your explanation of what you have done, and your results, here***

3. Estimate a homography transformation based on the matches, using `cv2.findHomography()`. Display the transformed outline of the first reference book cover image on the query image, to see how well they match.

    - We provide a function `draw_outline()` to help with the display, but you may need to edit it for your needs.
    - Try the 'least squre method' option to compute homography, and visualize the inliers by using `cv2.drawMatches()`. Explain your results.
    - Again, you don't need to compare results numerically at this stage. Comment on what you observe visually.

In [ ]:
src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
M, mask = cv2.findHomography(src_pts, dst_pts, 0)


draw_outline(img1, img2, M)
#plt.imshow(img2)
draw_inliers(img1,img2,kp1,kp2,good,mask)





# using cv2 standard method, see (3) above

# draw frame

# draw inliers



***Your explanation of results here***

Try the RANSAC option to compute homography. Change the RANSAC parameters, and explain your results. Print and analyze the inlier numbers.

In [ ]:
# Your code to display book location after RANSAC here

# using RANSAC
M1, mask1 = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)


draw_outline(img1, img2, M1)
#plt.imshow(img2)
draw_inliers(img1,img2,kp1,kp2,good,mask1)

# draw frame


# draw inliers


# inlier number
num_inliers = np.sum(mask1)
print("Number of inliers:", num_inliers)



***Your explanation of what you have tried, and results here***

6. Finally, try matching several different image pairs from the data provided, including at least one success and one failure case. For the failure case, test and explain what step in the feature matching has failed, and try to improve it. Display and discuss your findings.
    1. Hint 1: In general, the book covers should be the easiest to match, while the landmarks are the hardest.
    2. Hint 2: Explain why you chose each example shown, and what parameter settings were used.
    3. Hint 3: Possible failure points include the feature detector, the feature descriptor, the matching strategy, or a combination of these.

In [ ]:
# Your results for other image pairs here

***Your explanation of results here***

# Question 2: What am I looking at? (40%)


<!--One application of feature matching is image retrieval. The goal of image retrieval is, given a query image of an object, to find all images in a database containing the same object, and return the results in ranked order (like a Google search). This is a huge research area but we will implement a very basic version of the problem based on the small dataset provided.-->

In this question, the aim is to identify an "unknown" object depicted in a query image, by matching it to multiple reference images, and selecting the highest scoring match. Since we only have one reference image per object, there is at most one correct answer. This is useful for example if you want to automatically identify a book from a picture of its cover, or a painting or a geographic location from an unlabelled photograph of it.

The steps are as follows:

1. Select a set of reference images and their corresponding query images. 

    1. Hint 1: Start with the book covers, or just a subset of them.
    3. Hing 2: This question can require a lot of computation to run from start to finish, so cache intermediate results (e.g. feature descriptors) where you can.
    
2. Choose one query image corresponding to one of your reference images. Use RANSAC to match your query image to each reference image, and count the number of inlier matches found in each case. This will be the matching score for that image.

3. Identify the query object. This is the identity of the reference image with the highest match score, or "not in dataset" if the maximum score is below a threshold.

4. Repeat steps 2-3 for every query image and report the overall accuracy of your method (that is, the percentage of query images that were correctly matched in the dataset). Discussion of results should include both overall accuracy and individual failure cases.

    1. Hint 1: In case of failure, what ranking did the actual match receive? If we used a "top-k" accuracy measure, where a match is considered correct if it appears in the top k match scores, would that change the result?

<!--Code to implement this algorithm should mostly be written in a supporting file such as a2code.py. Call your code and display outputs in the notebook below.--> 


In [25]:
# Your code to iddntify query objects and measure search accuracy for data set here 
import os

ref_cache_dict = {}

for x in os.listdir('A2_smvs/book_covers/Reference'):
    if x in ref_cache_dict:
        ref_kp, ref_descriptors = ref_cache_dict[x]
    else:
        image = cv2.imread(f'A2_smvs/book_covers/Reference/{x}',0)
        ref_kp, ref_descriptors = orb.detectAndCompute(image,None)
        ref_cache_dict[x]=(ref_kp, ref_descriptors)

#query_chosen = cv2.imread("A2_smvs/book_covers/Query/091.jpg",0)
#query_chosen_kp,query_chosen_desc = orb.detectAndCompute(query_chosen,None)



def imageMatching(query_img,query_chosen_kp,query_chosen_desc):
    best_score = 0
    query_object = 0
    actual_match_score = 0
    dict = []
  
    for ref_img in os.listdir('A2_smvs/book_covers/Reference'):
        ref_kps,ref_descr = ref_cache_dict[ref_img]
        
        matches = bf.knnMatch(ref_descr,query_chosen_desc,2)
        good = []

        for m,n in matches:
            if m.distance < 0.8*n.distance:
                good.append(m)
        if len(good)>=4:
            src_pts = np.float32([ref_kps[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([query_chosen_kp[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            
            #if ref_img == os.path.basename(f'A2_smvs/book_covers/Query/{query_img}'):
              #  actual_match_score = np.sum(mask)

            num_inliers = np.sum(mask)
            values = (ref_img,num_inliers)
            
            dict.append(values)
        
            #if num_inliers > best_score:
             #   best_score = num_inliers
              #  query_object = ref_img


    dict.sort(key=lambda i:i[1],reverse=True) 
   


    return dict



no_of_matches = 0
failures = 0

query_dict = {}

for query_img in os.listdir('A2_smvs/book_covers/Query'):
    query_chosen = cv2.imread(f'A2_smvs/book_covers/Query/{query_img}',0)
    query_chosen_kp,query_chosen_desc = orb.detectAndCompute(query_chosen,None)

    query_dict[query_img]= imageMatching(query_img,query_chosen_kp,query_chosen_desc)

    if query_dict[query_img][0][0] == os.path.basename(f'A2_smvs/book_covers/Query/{query_img}'):
        no_of_matches=no_of_matches+1
    else:
        failures = failures + 1 
       # print("best:",test2)
        #print("actual:",test3)
   

#print(failures)

accuracy =  no_of_matches/(failures+no_of_matches)

print(accuracy)








0.6633663366336634


***Your explanation of what you have done, and your results, here***

5. Choose some extra query images of objects that do not occur in the reference dataset. Repeat step 4 with these images added to your query set. Accuracy is now measured by the percentage of query images correctly identified in the dataset, or correctly identified as not occurring in the dataset. Report how accuracy is altered by including these queries, and any changes you have made to improve performance.

In [ ]:
# Your code to run extra queries and display results here

***Your explanation of results and any changes made here***


6. Repeat step 4 and 5 for at least one other set of reference images from museum_paintings or landmarks, and compare the accuracy obtained. Analyse both your overall result and individual image matches to diagnose where problems are occurring, and what you could do to improve performance. Test at least one of your proposed improvements and report its effect on accuracy.


In [ ]:
# Your code to search images and display results here

***Your description of what you have done, and explanation of results, here***